In [1]:
import arcgis
from arcgis.gis import GIS
import arcgis.geocoding as geocoding
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import osmapi
import csv
import pandas as pd

In [5]:
my_gis = GIS()

In [3]:
hello_map = my_gis.map("Санкт-Петербург", zoomlevel=15)

In [4]:
hello_map.basemap = 'osm'

In [5]:
class Adres_read():
    def __init__(self, adress):
        self.__geolocator = Nominatim(user_agent="specify_your_app_name_here")
        self.__location = self.__geolocator.geocode(adress)
        # self.__location_place = (self.__location.latitude, self.__location.longitude)

        self.__bounding = self.__location.raw['boundingbox']

    def get_points(self):
        return self.__bounding

In [6]:
class Osm_reader():
#     def __init__(self, min_lon, min_lat, max_lon, max_lat):
    def __init__(self, lil):
#         print(lil)
        self.__api = osmapi.OsmApi()
#         self.__tmp = self.__api.Map(min_lon, min_lat, max_lon, max_lat)
        self.__tmp = self.__api.Map(float(lil[2]), float(lil[0]), float(lil[3]), float(lil[1]))
        self.__ret = []

        for e in self.__tmp:
            try:
                try:
                    try:
                        type_ = e['type']
                        timestamp = e['data']['timestamp']
                        id = e['data']['id']
                        lat = e['data']['lat']
                        lon = e['data']['lon']
                        uid = e['data']['uid']
                        user = str(e['data']['user'])
                        tag = e['data']['tag']
                        delta_days = (osmapi.datetime.today() - timestamp).days

                        if type_ == 'node':
                            self.__ret.append([id, lat, lon, uid, timestamp, user])

                    except KeyError:
                        pass
                except IndexError:
                    pass
            except UnicodeEncodeError as err:
                print(err, str(e) + '\n')

    def create_file(self):
        with open('dumps.csv', "w", newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in self.__ret:
#                 print(line)
                writer.writerow(line)
                
            csv_file.close()

    def get_values(self):
        return self.__ret

In [9]:
def find_addr(hello_map, g):
#     try:
    hello_map.draw(g)
    geocoded = geocoding.reverse_geocode(g)
    address = geocoded['address']['Match_addr']
    adres_read = Adres_read(address)
#     print(adres_read.get_points())

    try:
        print('creating')
        a = Osm_reader(adres_read.get_points())
        a.create_file()
        print('file created')
    except:
        print("beda")


hello_map.on_click(find_addr)
# hello_map.on_click(add_costil())
hello_map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True, zoom=14.0)

creating
file created
creating
file created
creating
file created
creating
file created
creating
file created
creating
file created
creating
beda
creating
beda
creating
beda
creating
beda


In [10]:
# def add_costil():
kostil = pd.read_csv('dumps.csv')
kostil.columns = ['id', 'lat', 'lon', 'uid', 'timestamp', 'user']
if len(kostil) > 999:
    kostil = kostil[0:1000]
    hello_map.add_layer(my_gis.content.import_data(kostil))
else:
    hello_map.add_layer(my_gis.content.import_data(kostil))

In [11]:
# hello_map.add_layer(my_gis.content.import_data(kostil))